In [1]:
# Install Pytorch & other libraries
%pip install "torch==2.4.0" tensorboard pillow
 
# Install Hugging Face libraries
%pip install  --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen-vl-utils"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 5.5 MB/s eta 0:00:0000:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 30.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 129.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 136.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 149.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 20.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 59.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 61.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 77.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 42.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 60.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━

In [2]:
import torch

# GPU가 Flash Attention을 지원하는지 확인
assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'

# Flash Attention 설치
!pip install ninja packaging
!pip install flash-attn --no-build-isolation --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 3.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 19.5 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [179]:
system_message = """당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-------------------------
{search_result}"""

In [180]:
from datasets import load_dataset

# sample["search_result"] 리스트를 원하는 형식의 문자열로 변환하는 함수
def format_search_result(search_results):
    return "\n-------------------------\n".join([f"문서{idx + 1}: {result}" for idx, result in enumerate(search_results)])

# 데이터셋을 OpenAI 메시지 형식으로 변환하는 함수      
def format_data(sample):
    formatted_search_result = format_search_result(sample["search_result"])
    return {
        "messages": [
            {
                "role": "system",
                "content": system_message.format(search_result=formatted_search_result),
            },
            {
                "role": "user",
                "content": sample["question"],
            },
            {
                "role": "assistant",
                "content": sample["answer"]
            },
        ],
    }

# 허브에서 데이터셋 로드
dataset = load_dataset("iamjoon/klue-mrc-ko-rag-dataset", split="train")

# 데이터셋을 OpenAI 메시지 형식으로 변환
dataset = [format_data(sample) for sample in dataset]

print(dataset[345]["messages"])

[{'role': 'system', 'content': '당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n-------------------------\n문서1: 새롭게 창출되어야 할 시장 중 중요한 부분이 있거나 급한 경우 또는 필요하게 되는 경우가 있으면 자유 진입이 중요시되어야 하나, 신규 진입을 할 때 장애물이 생기게 되면서 사업할 때 막히는 경우가 있다. 단적인 예로 보면 지역 난방을 개별 난방으로 전환되는 사업이라던가 24시간 자유자재 생활하게 되어 있어야 할 조건이 불리한 경우, 2019년 한일 무역 분쟁 이후 일본으로부터 신규 원자재 도입 시 특정된 업체들 에 직접 납품하게 될 재료가 필요하게 되면 당연히 넣어야 하는 조건이 있어야 한다. 그래도 안된다면 제3국으로 수출시킬 때 일본의 동의 없이 일본이 아닌 업체에 조달해야 하는 대체재를 바꿀 수밖에 없다. 하지만 이와 같은 원자재와 같이 긴요하게 공급되어야 할 필요 재료들을 직접 반입시킬 때 막히는 형태를 가진 나비 효과와도 이와 비슷한 연관성이 있다. 그 외에도 같은 조건만 제약된 형태로 생활하게 되어 있는 식으로만 사용하여도 진입 장벽이 적용될 수도 있게 된다. 끝으로 게임에서도 진입 장벽을 

In [181]:
# 현재 전체 데이터 개수
print(f"현재 전체 데이터 개수: {len(dataset)}")

# 테스트용으로 50% 따로 저장 
test_size = int(len(dataset) * 0.5)
test_dataset = dataset[-test_size:]  # 뒤에서 50%

# 테스트셋 저장
from datasets import Dataset
test_dataset = Dataset.from_list(test_dataset)
test_dataset.save_to_disk('test_dataset')
print(f"테스트 데이터 개수: {len(test_dataset)}")

현재 전체 데이터 개수: 1884


Saving the dataset (0/1 shards):   0%|          | 0/942 [00:00<?, ? examples/s]

테스트 데이터 개수: 942


In [182]:
# 앞의 데이터 중 20%를 학습셋으로 사용
dataset = dataset[:int(len(dataset) * 0.2)]

print(f"데이터 개수: {len(dataset)}")

데이터 개수: 376


In [128]:
# list 형태의 dataset을 Dataset 객체로 변환
from datasets import Dataset

dataset = Dataset.from_list(dataset)

In [129]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-7B-Instruct" 

# BitsAndBytes 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,                             # 4비트 양자화 사용
   bnb_4bit_use_double_quant=True,               # 이중 양자화 사용으로 메모리 추가 절약
   bnb_4bit_quant_type="nf4",                    # 4비트 양자화 타입 설정(normalized float 4)
   bnb_4bit_compute_dtype=torch.bfloat16         # 연산 시 bfloat16 타입 사용
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [130]:
# 템플릿 적용
text = tokenizer.apply_chat_template(
    dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-------------------------
문서1: 열대우림 '장마전선'은 벵골만과 서북태평양에서 동아시아 몬순의 하위시스템으로 조성된다. '장마전선'의 북진 움직임은 아열대 능선이 발달한 데 영향을 받는다. 이 북쪽으로 이동하는 준정전선은 남한에서 '장마'라고 불리며, 주요 강수 기간을 나타낸다. 창마전선'은 한반도를 통과하는 데 약 4~5주가 걸린다. 이러한 느린 움직임은 매년 6월말과 7월에 한반도 전체에 많은 양의 여름 강우량을 발생시킨다. 최근 들어 '창마전선'은 7월 말부터 8월 초까지 다양한 규모의 폭풍우와 함께 폭우가 쏟아지면서 한반도를 통과하는 데 3주도 채 걸리지 않는 등 빠르게 움직이는 경향을 보였다. '창마' 이후 더 극한의 날씨와 국지적인 폭우가 발생하고 있다는 뜻이다. 잠열 방출에 의해 강하게 변형된 바로크린 교란에서 비롯된 초여름의 '창마' 비의 역학관계는 여전히 제대로 파악되지 않고 있다. 가을 창마로 부를 수 있는 또 다른 '창마' 유형도 있다. 이는 물론 기상청의 공식 용어는 아니다. 그러나 최근의 기후 변화로 인해 '낙하 창마'라는 용어가 생겨났다. '낙하 창마'

In [104]:
from peft import LoraConfig
 
# QLoRA 논문 및 Sebastian Raschka 실험에 기반한 LoRA Conifg
peft_config = LoraConfig(
        lora_alpha=8,
        lora_dropout=0.05,
        r=6,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [105]:
# from transformers import TrainingArguments
from trl import SFTConfig

args = SFTConfig(
    output_dir="qwen2-7b-rag-ko",     # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=2,           # 장치당 학습 배치 크기
    gradient_accumulation_steps=2,           # 역전파/가중치 업데이트 전 누적할 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 그래디언트 체크포인팅 사용
    optim="adamw_torch_fused",               # 퓨즈드 AdamW 옵티마이저 사용
    logging_steps=10,                        # 10스텝마다 로그 기록
    save_strategy="steps",                   # 특정 스텝마다 체크포인트 저장
    save_steps=50,
    bf16=True,                              # bfloat16 정밀도 사용
    tf32=True,                              # tf32 정밀도 사용
    learning_rate=1e-4,                     # 학습률 (QLoRA 논문 기반)
    max_grad_norm=0.3,                      # 최대 그래디언트 노름 (QLoRA 논문 기반)
    warmup_ratio=0.03,                      # 워밍업 비율 (QLoRA 논문 기반)
    lr_scheduler_type="constant",           # 고정 학습률 스케줄러 사용
    push_to_hub=False,                      # 허브에 모델 업로드 안 함
    report_to="tensorboard",                # 텐서보드에 메트릭 기록
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True}
)

In [131]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }
    
    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)
        
        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()
        
        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        
        # 레이블 초기화
        labels = [-100] * len(input_ids)
        
        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"
        
        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)
        
        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and 
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):
                
                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and 
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):
                    
                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)
                    
                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and 
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1
                    
                    i = current_pos
                
            i += 1
        
        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)
    
    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])
    
    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])
        
        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)
    
    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)
    
    return new_batch

def print_tokens_and_labels(batch):
    input_ids = batch["input_ids"][0].tolist()
    labels = batch["labels"][0].tolist()
    
    print("\n토큰과 레이블 비교:")
    print(f"{'Token ID':<10} {'Token':<30} {'Label':<10}")
    print("-" * 50)
    
    for token_id, label in zip(input_ids, labels):
        token = tokenizer.decode([token_id])
        label_str = str(label) if label != -100 else "-100"
        print(f"{token_id:<10} {token:<30} {label_str:<10}")

In [132]:
# collate_fn 테스트 (배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 4757])
어텐션 마스크 형태: torch.Size([1, 4757])
레이블 형태: torch.Size([1, 4757])


In [133]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0].tolist())

입력에 대한 정수 인코딩 결과:
[151644, 8948, 198, 64795, 82528, 33704, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 138520, 19391, 143604, 129264, 130650, 382, 13146, 48431, 20401, 66790, 29326, 131193, 17877, 125686, 125548, 139713, 624, 16, 13, 138520, 53680, 85322, 77226, 98801, 18411, 81718, 144059, 42039, 143604, 16186, 139713, 624, 17, 13, 85322, 77226, 98801, 19391, 130768, 130213, 17877, 143604, 16186, 125476, 34395, 53900, 21329, 95577, 139713, 624, 18, 13, 138520, 19391, 128605, 143603, 12802, 85322, 77226, 98801, 19391, 130671, 32290, 85322, 77226, 98801, 126377, 330, 33883, 64795, 138520, 93, 19391, 128605, 130213, 12802, 136673, 1189, 5140, 45881, 34395, 143604, 16186, 139713, 624, 19, 13, 143604, 47836, 53618, 142976, 139236, 18411, 142616, 82190, 53435, 40853, 129549, 53435, 125068, 17877, 140174, 128836, 32290, 5140, 240, 97, 19391, 36330, 250, 125746, 16560, 23084, 126402, 83634, 17380, 94613, 139236, 84621, 47324, 18411, 129624, 20487, 139713, 13, 95617, 18411, 129901, 26698

In [134]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0].tolist())

레이블에 대한 정수 인코딩 결과:
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

In [110]:
from trl import SFTTrainer

max_seq_length = 8192  # 모델과 데이터셋 패킹을 위한 최대 시퀀스 길이

trainer = SFTTrainer(
    model=model,
    args=args,
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이 설정
    train_dataset=dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [111]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
10,0.570300
20,0.501400
30,0.388700
40,0.399800
50,0.470000
60,0.475100
70,0.423000
80,0.458300
90,0.442900
100,0.412300


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

KeyboardInterrupt: 

In [112]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [113]:
%pwd

'/workspace'

In [114]:
%ls -al

total 4107
drwxrwxrwx 4 root root 2011095 Nov  2 13:43 ./
drwxr-xr-x 1 root root     124 Nov  2 11:47 ../
drwxrwxrwx 2 root root    7200 Nov  2 11:48 .ipynb_checkpoints/
-rw-rw-rw- 1 root root  175511 Nov  2 13:43 Untitled.ipynb
drwxrwxrwx 6 root root 2011078 Nov  2 13:42 qwen2-7b-rag-ko/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [115]:
!wget https://raw.githubusercontent.com/ukairia777/LLM-Finetuning-tutorial/main/merge.py

--2024-11-02 13:45:09--  https://raw.githubusercontent.com/ukairia777/LLM-Finetuning-tutorial/main/merge.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


200 OK
Length: 1351 (1.3K) [text/plain]
Saving to: ‘merge.py’

merge.py            100%[===================>]   1.32K  --.-KB/s    in 0s      

2024-11-02 13:45:09 (113 MB/s) - ‘merge.py’ saved [1351/1351]



In [183]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import  AutoTokenizer, pipeline

base_model_id = "Qwen/Qwen2-7B-Instruct"
peft_model_id = "qwen2-7b-rag-ko/checkpoint-150"
 
# Load Model with PEFT adapter
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
# get token id for end of conversation
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [184]:
# 테스트 데이터
text = tokenizer.apply_chat_template(
    test_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
text = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
print(text)

<|im_start|>system
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문~에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어서 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어서 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]이라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
-------------------------
문서1: 참신하고 새로운 아이디어로 무장한 관광기업들을 대상으로 한 크라우드 펀딩이 점차 활기를 띠고 있다. 크라우드 펀딩은 자금이 필요한 기업이 온라인 플랫폼을 기반으로 불특정 다수로부터 투자를 받는 방식으로, 문화체육관광부(장관 박양우)와 한국관광공사(사장 안영배)에서는 2017년부터 관광기업을 대상으로 ‘관광 크라우드 펀딩 지원사업’을 추진하고 있다. 올해엔 등록 프로젝트 91개 중 82개가 펀딩에 성공해 총 26억 원의 투자금을 유치하는 성과를 거뒀다. 이 실적은 75개 프로젝트 중 53개가 성공해 총 12억 원을 확보한 전년에 비해 참여 프로젝트 수 21.3%, 펀딩성공률 19.5%p, 총 투자액 2배 이상이 상승한 것이다. 한편 문체부와 공사는 올해 탁월한 성과를 거둔 8개 우수기업을 선정, 12월 23일(월) 서울 서촌창작소에서 시상식을 가졌다. 2019년 대상은 ‘스테이폴리오(대표 이상묵)’가 차지했다. 스테이폴리오는 서울 서촌지역 한옥 재생 프로젝트 ‘서촌유희’ 운영을 위한 증권형* 크라우드 펀딩을 등록한 후 열흘이 채 되지 않는 기간 동

In [185]:
prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [186]:
len(prompt_lst)

942

## 기본 모델

In [187]:
def test_inference(prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()
 
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


    response:
구석기 시대와 철기 시대의 도구 제작 방식에는 여러 가지 차이가 있습니다.

구석기 시대의 도구 제작 방식은 주로 간접떼기 방식을 사용했습니다. 이 방식은 격지나 돌날을 이용해 도구를 만드는 것으로, 주먹도끼보다 훨씬 작고 더 세밀한 도구를 만들 수 있었습니다. 슴베찌르개는 이 시대의 대표적인 도구로, 자루를 달아서 짐승을 사냥하는 창이나 전쟁 무기 또는 가죽에 구멍을 뚫는 연장 등으로 사용되었습니다. 슴베찌르개는 자루를 달아서 사용하는 것이 특징이며, 양 옆의 마름모꼴 모서리 주변에 잔손질을 해 위쪽 끝이 날카로워 사용하기에 적합했습니다.

반면에 철기 시대의 도구 제작 방식은 주로 직접떼기 방식을 사용하였습니다. 이 방식은 석기를 직접 떼어 사용하는 방식으로, 전기에는 외날찍개와 같이 직접떼기로 한 면만 떼어 낸 석기, 중기에는 주먹도끼와 같이 두 면을 떼어 낸 쌍날찍개가 사용되었습니다. 철기 시대의 도구 제작 방식은 더 복잡하고 세밀한 작업을 필요로 하며, 이로 인해 더욱 효율적이고 효과적인 도구를 만들 수 있었습니다.

따라서 구석기 시대와 철기 시대의 도구 제작 방식의 주요 차이점은 제작 방식과 도구의 복잡성 및 효율성에 있습니다.
    label:

구석기 시대와 철기 시대의 도구 제작 방식에는 여러 가지 차이가 있습니다.

구석기 시대의 도구는 주로 돌을 깨뜨려 사용하기 편리하도록 만든 뗀석기(타제석기)였습니다. 초기 구석기 시대에는 외날찍개와 같이 한 면만 떼어 낸 석기가 사용되었고, 중기에는 주먹도끼와 같이 두 면을 떼어 낸 쌍날찍개가 사용되었습니다. 후기 구석기 시대에는 격지나 돌날의 양쪽을 단단한 뼈나 뿔로 눌러 떼어 도구를 만드는 간접떼기 방식이 사용되었습니다. 대표적인 도구로는 슴베찌르개가 있으며, 이는 자루를 달아서 사냥이나 전쟁 무기 등으로 사용되었습니다 [[ref1]].

반면 철기 시대에는 금속을 사용한 도구와 무기가 제작되었습니다. 예를 들어, 철기 시대의 동검은 칼과 손잡이, 칼끝 장식이 별개로 만들

## 학습 모델

In [190]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-150"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [ ]:
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


    response:
구석기 시대와 철기 시대의 도구 제작 방식에는 여러 가지 차이가 있습니다.

구석기 시대에는 주로 뗀석기(타제석기)를 사용하여 도구를 제작했습니다. 이는 주로 돌을 깨뜨려 사용하기 편리하도록 만든 방식입니다. 예를 들어, 슴베찌르개는 격지나 돌날을 이용해 만들었으며, 주로 돌을 깨뜨려 사용하기 편리하도록 제작되었습니다. 슴베찌르개는 자루를 달아서 짐승을 사냥하는 창이나 전쟁 무기 또는 가죽에 구멍을 뚫는 연장 등으로 사용되었습니다.

반면에 철기 시대에는 철이나 다른 금속을 사용하여 도구를 제작하는 방식이 주로 사용되었습니다. 철기 시대의 도구 제작 방식은 주로 철이나 다른 금속을 가공하여 원하는 형태로 만들어내는 것입니다. 예를 들어, 동검은 전형적인 세형동검으로 칼 끝이 예리하고 칼몸 끝까지 등날이 세워져 있으며, 칼자루 끝에는 물새 두 마리가 머리를 돌리고 서로 바라보는 모습을 하고 있습니다. 이는 전에 평양부근에서 출토된 적이 있는 것으로 북방지역 청동기 문화와의 연관성을 나타냅니다.

따라서, 구석기 시대와 철기 시대의 도구 제작 방식의 주요 차이점은 사용되는 재료와 제작 방식에 있습니다. 구석기 시대에는 주로 돌을 사용하여 뗀석기를 사용하는 방식으로 제작하였으며, 철기 시대에는 철이나 다른 금속을 사용하여 가공하여 원하는 형태로 제작하는 방식을 사용하였습니다. [[ref1]], [[ref2]]
    label:

구석기 시대와 철기 시대의 도구 제작 방식에는 여러 가지 차이가 있습니다.

구석기 시대의 도구는 주로 돌을 깨뜨려 사용하기 편리하도록 만든 뗀석기(타제석기)였습니다. 초기 구석기 시대에는 외날찍개와 같이 한 면만 떼어 낸 석기가 사용되었고, 중기에는 주먹도끼와 같이 두 면을 떼어 낸 쌍날찍개가 사용되었습니다. 후기 구석기 시대에는 격지나 돌날의 양쪽을 단단한 뼈나 뿔로 눌러 떼어 도구를 만드는 간접떼기 방식이 사용되었습니다. 대표적인 도구로는 슴베찌르개가 있으며, 이는 자루를 달아서 사냥이나 전쟁 무기 등으로 사